In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

In [6]:
# transmuting old data that I can't rerun on mbp
options = ['blingfire','reweightedblingfire','nltk','reweightednltk','spacysm','reweightedspacysm','spacylg','reweightedspacylg','spacytrf','reweightedspacytrf']

for name in options:
    postdata = pd.read_json(f'./data/postdatalinesvectors{name}.json',orient='records', lines=True)

    train_set, testval_set = train_test_split(postdata, test_size=0.3, random_state=99)

    test_set, val_set = train_test_split(testval_set, test_size=0.33, random_state=99)

    train_set['split'] = 'train'
    test_set['split'] = 'test'
    val_set['split'] = 'val'

    total = pd.concat([train_set,test_set,val_set])

    total.to_json(f'../data/vectorData/{name}Vectors.json', orient='records',lines=True)


In [7]:
options = ['blingfire','reweightedblingfire','nltk','reweightednltk','spacysm','reweightedspacysm','spacylg','reweightedspacylg','spacytrf','reweightedspacytrf']

for name in options:
    df = pd.read_json(f'../data/vectorData/{name}Vectors.json', orient='records', lines=True)

    (df['gender'] == 'm') & (df['split'] == 'train')

    X_train = df.loc[(df['split'] == 'train'), [str(i) for i in range(38)]]
    X_test = df.loc[(df['split'] == 'test'), [str(i) for i in range(38)]]
    X_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), [str(i) for i in range(38)]]
    X_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), [str(i) for i in range(38)]]

    y_train = df.loc[(df['split'] == 'train'), ['label']]
    # y_test = df.loc[(df['split'] == 'test'), ['label']]
    # y_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['label']]
    # y_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['label']]

    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    X_testm = scaler.transform(X_testm)
    X_testf = scaler.transform(X_testf)

    y_train = y_train.values.ravel()

    param_grid = {'C': np.logspace(-3, 3, 7),  
                'gamma': np.logspace(-3, 3, 7), 
                'kernel': ['rbf']}  

    cv = 5
    scoring = 'accuracy'

    grid = GridSearchCV(SVC(random_state=99, probability=True, class_weight='balanced'), param_grid, scoring=scoring, cv=cv, n_jobs=-1)
    grid.fit(X_train,y_train)

    y_pred = grid.predict(X_test)
    y_prob = grid.predict_proba(X_test)[:,1]

    testSet = df.loc[(df['split'] == 'test')]

    testSet['prediction'] = y_pred
    testSet['probability'] = y_prob

    testSet.to_json(f'../data/predictionData/{name}Pred.json',orient='records',lines=True)

C:\Users\bramb\AppData\Local\Temp\ipykernel_15216\2513260686.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['prediction'] = y_pred
C:\Users\bramb\AppData\Local\Temp\ipykernel_15216\2513260686.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['probability'] = y_prob
C:\Users\bramb\AppData\Local\Temp\ipykernel_15216\2513260686.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 